# CPE - TP ML 2024/25: Supervised Learning
## Classification of MNIST data
### Contents : Carole Lartizien


### Family Name and surname :

## - Import some usefull libraries

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pylab import *

from sklearn.datasets import fetch_openml
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn import datasets
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

## - Define some usefull functions

In [ ]:
#for sklearn version >0.22, use ConfusionMatrixDisplay
#Else, use the following function to plot confusion matrix

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


## - Import the MNIST dataset
### Deux méthodes, directement par fetch_openml ou telechargement des donnees puis ouverture avec loadmat

In [ ]:
# Load data from https://www.openml.org/d/554
X, Y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = np.array(X)
Y = np.array(Y)

print(X.shape)
print(Y.shape)
# Attention, Y est un str par defaut, le convertir en int
Y=Y.astype(int)

In [ ]:
# download MNIST dataset from https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat
from scipy.io import loadmat
mnist = loadmat("mnist-original.mat")
X= mnist["data"].T
Y= mnist["label"][0]


print(X.shape)
print(Y.shape)

### **Question** : À quoi correspond ici le '784' dans 'mnist_784' ?

## - Format the training and validation database

In [ ]:
#Selection de 5000 points d'entrainements parmi les 60000 premiers exemples de MNIST
np.random.seed(seed=10)

ind_train=np.random.choice(np.arange(60000)+1, 5000, replace = False)

print(f"Nombres d'indices selectionnés : {ind_train.shape}")
print(f"Indice maximum : {max(ind_train)}")
print(f"indice minimum : {min(ind_train)}")

Xtrain=X[ind_train]
print(f"Nouvelle forme Xtrain : {Xtrain.shape}")

Ytrain=Y[ind_train]
print(f"Nouvelle forme Ytrain : {Ytrain.shape}")


sum_ = 0
print(f"Valeur max Ytrain : {max(Ytrain)}")
print(f"Valeur min Ytrain : {min(Ytrain)}")
for k in range(0, 10):
    print(f"Nombre d'images d'entrainement de label {k} : {(Ytrain == k).sum()}")
    sum_ += ((Ytrain == k).sum())
print(f"Nombre total d'images d'entrainement: {sum_}")

#Construction de la base de test : 10000 derniers exemples de MNIST
sum_ = 0
Xtest = X[60000:]
Ytest = Y[60000:]

for k in range(0, 10):
    print(f"Nombre d'images de test de label {k}: {(Ytest == k).sum()}")
    sum_ += ((Ytest == k).sum())
print(f"Nombre total d'images de test : {sum_}")

#### Une autre manière de faire :

In [ ]:
for k in range(0, 10):
    i=np.where(Ytest == k)
    print(f"Nombre d'images de test de label {k} : {size(i)}")

In [ ]:
print(X.shape, Y.shape)
print(Xtrain.shape, Ytrain.shape)
print(Xtest.shape, Ytest.shape)

#Normalisation des données
sc = StandardScaler().fit(Xtrain)
Xtrain = sc.transform(Xtrain)
Xtest = sc.transform(Xtest)

#### Question : Pourquoi normalise-t-on les données ? Que fait cette normalisation ?

## - Visualisation des données

In [ ]:
indices_img=[11450, 12585, 9520, 4587, 2351]

for ind_img in indices_img:
    im=X[ind_img]
    im = im.reshape((28,28))
    plt.figure()
    plt.imshow(im, cmap="Greys_r")
    plt.title(Y[ind_img])


## - Comparaison des performances de différents classifieurs

#### -  Entrainez et mesurez les performances des modèles suivants:
- SVM : linéaire et non -linéaire (RBF)
- Decision tree
- Random forest
- Régression logistique
.....

#### - Pour chaque modèle, lister et optimiser les hyperparamètres. Afficher les meilleures performances.

#### Astuce : utilisez la fonction GridSearchCV

--------------------
### SVM linéaire (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html):
--------------------
#### Lister les hyperparamètres ici:
- XX



In [ ]:
from sklearn import svm
parameters = {'C':[0.1, 1, 10, 100]}
svc = svm.SVC(kernel="linear")
svc.get_params()

In [ ]:
clf_SVML = GridSearchCV(svc, parameters, cv=5)
clf_SVML.fit(Xtrain, Ytrain)

In [ ]:
print(clf_SVML.best_score_)
print(clf_SVML.best_estimator_.C)
clf_SVML.score(Xtest, Ytest)

In [ ]:
Ytest_pred=clf_SVML.predict(Xtest)
print("Classification report for classifier %s:\n%s\n"
      % (clf_SVML, metrics.classification_report(Ytest, Ytest_pred)))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
display = ConfusionMatrixDisplay.from_estimator(clf_SVML, Xtest, Ytest)
plt.close()  # close the 1st small figure generated by ConfusionMatrixDisplay
display.plot(ax=ax)

fig, ax = plt.subplots(figsize=(10, 10))
display = ConfusionMatrixDisplay.from_estimator(clf_SVML, Xtest, Ytest, normalize='true')
plt.close()
display.plot(ax=ax)

In [ ]:
#For sklearn <0.22
digit_names=np.asarray(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
np.set_printoptions(precision=1)
plot_confusion_matrix(Ytest, Ytest_pred, classes = digit_names,title='Confusion matrix, without normalization')
plot_confusion_matrix(Ytest, Ytest_pred, classes=digit_names, normalize=True, title='Normalized confusion matrix')
plt.show()

*************************************
### Non linear SVM with rbf kernel
*************************************
#### List of hyperparameters:
- XX
- XX

In [ ]:
parameters = {'gamma':[0.001, 0.01, 0.1], 'C':[1, 10]}
svc = svm.SVC(kernel="rbf")
svc.get_params()

### A vous de jouer

*************************************
### Logistic regression
*************************************
#### List of hyperparameters:
- XX
- XX


### A vous de jouer

In [ ]:
from sklearn.linear_model import LogisticRegression

*************************************
### Decision tree (https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
*************************************
#### List of hyperparameters:
- XX

In [ ]:
# Decision Tree
from sklearn import tree

In [ ]:
plt.figure(figsize=(20, 20))
tree.plot_tree(clf_DT.best_estimator_)

*************************************
### Random forest (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
*************************************
#### List of hyperparameters:
- n_estimators= Number of trees
- criterion{“gini”, “entropy”}, default=”gini”
- splitter{“best”, “random”}, default=”best”
- max_depthint, default=None
- min_samples_splitint or float, default=2
- max_features

In [ ]:
from sklearn.ensemble import RandomForestClassifier

*************************************
### Neural network - perceptron
*************************************
#### List of hyperparameters:
-   mlp = MLPClassifier(
    hidden_layer_sizes=(40,),
    max_iter=8,
    alpha=1e-4,
    solver="sgd",
    verbose=10,
    random_state=1,
    learning_rate_init=0.2,
- XX


In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
parameters = {'max_iter':[10, 100]}
MLP=MLPClassifier(random_state=1, hidden_layer_sizes=(40,30))
MLP.get_params()

## Feature extraction (optional)

Extract other feature types from the original MNIST data, eg
HOG features (see https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html)

Estimate the best performance achieved with this new feature set. How does it compare to the performance achieved with the original grey level features?

https://github.com/OSSpk/Handwritten-Digits-Classification-Using-KNN-Multiclass_Perceptron-SVM/blob/master/Code/multiclass_classification.py

In [ ]:
from skimage.feature import hog
from skimage import data, exposure
from skimage.transform import rescale, resize, downscale_local_mean

ind_img=11450
im=X[ind_img]
print(Y[ind_img])
im = im.reshape((28,28))
print(im.shape)

im_res=resize(im,(112,112), anti_aliasing=True)
image = im_res

print(image.shape)
plt.imshow(image, cmap="Greys_r")

fd, hog_image = hog(image, orientations=4, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=True, multichannel=False)
fd.shape

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)
ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()

In [ ]:
X_hog=np.zeros((X.shape[0],256))
toto=np.zeros(256)
for k in range (0, 100):
#for k in range (0, X.shape[0]):
    im=X[k]
    im = im.reshape((28,28))
    im_res=resize(im,(112,112), anti_aliasing=True)
    fd, hog_image = hog(im_res, orientations=4, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=True, multichannel=False)
    X_hog[k,:]=fd[:]